<a href="https://colab.research.google.com/github/migdashn/Deep-Learning-Projects/blob/main/Colorozation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils import shuffle
from glob import glob

data processing

In [ ]:
#TODO

Generator

In [ ]:
def Generator():
    inputs = Input(shape=(256,256,3), name="InputLayer")
    down_stack = [downsample(64, apply_batchnorm=False),
        downsample(128),
        downsample(256),
        downsample(512),
        downsample(512),
        downsample(512),
        downsample(512),]
    encoding = downsample(512)

    up_stack = [upsample(512, apply_dropout=True),
        upsample(512, apply_dropout=True),
        upsample(512, apply_dropout=True),
        upsample(512),
        upsample(256),
        upsample(128),
        upsample(64),]

    x = inputs 
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    
    x = encoding(x)

    skips = reversed(skips)
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = concatenate([x, skip])
    
    init = RandomNormal(stddev=0.02)
    out = Conv2DTranspose(3, kernel_size=4, strides=2, kernel_initializer=init, activation='tanh', padding='same')

    out = out(x)

    gen = Model(inputs=inputs, outputs=out, name="Generator")
    return gen


Encoder Block

In [ ]:
def downsample(filters, apply_batchnorm=True):

    model = Sequential(
        Conv2D(filters, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False),
    )

    if apply_batchnorm:
        model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    return model

Decoder BLock

In [ ]:
def upsample(filters, apply_dropout=False):

    model = Sequential()
    model.add(Conv2DTranspose(
        filters,
        kernel_size=4,
        strides=2,
        padding='same',
        kernel_initializer='he_normal',
        use_bias=False))
    model.add(BatchNormalization())
    
    if apply_dropout:
        model.add(Dropout(0.5))
    
    model.add(ReLU())

    return model

Discriminator Model

In [ ]:
def Discriminator():
    init = RandomNormal(stddev=0.02)

    image = Input(shape=(256,256,3), name="ImageInput")
    target = Input(shape=(256,256,3), name="TargetInput")
    x = concatenate([image, target])

    x = downsample(64, apply_batchnorm=False)(x)
    x = downsample(128)(x)
    x = downsample(512)(x)

    x = ZeroPadding2D()(x)
    x = Conv2D(512, kernel_size=4, strides=1, kernel_initializer=init, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = ZeroPadding2D()(x)

    x = Conv2D(1, kernel_size=4, kernel_initializer=init)(x)

    model = Model(
        inputs=[image, target],
        outputs=x,
        name="Discriminator"
    )

    return model